In [29]:
import pandas as pd
import numpy as np
import os
import warnings
import datetime

In [30]:
warnings.filterwarnings("ignore")

#set working dir
os.chdir('C:/Users/Jackson/Documents/Data Science/Sephora Scraper_Attempts')
#read in review data
review_db = pd.read_csv('sephora_review_db.csv')

#remove products that arent foundations
not_foundations = ["Capture Dreamskin Fresh & Perfect Cushion Broad Spectrum SPF 50 Refill",
                  "Double Wear Stay-In-Place Foundation Pump",
                  "Teint Idole Ultra Longwear Cushion Foundation SPF 50 Refill",
                  "Chromatic Mix - Oil Base",
                   "Chromatic Mix - Water Base"
                  "Liquid Foundation Pump"]

df = review_db[~(review_db["name"].isin(not_foundations))]

remove any products that Sephora categories in the foundation category but is not a foundation.

In [54]:
#Add Product Info as specified by Sephora

#Coverage
df['coverage'] = [i.split("Coverage:", 1)[1].split(" ")[1] if "Coverage:" in i else '' for i in df["description"]]
df['coverage'] = [i.split('Skin')[0].split('Finish')[0] for i in df['coverage']]
df['coverage'] = [i if i in (['Medium', 'Light', 'Full']) else float("Nan") for i in df["coverage"]]

#Finish 
df['finish'] = [i.split('Finish:', 1)[1].split(' ')[1].split('x')[0].split('SPF')[0].split('What')[0].split('Formulation')[0] if "Finish:" in i else '' for i in df['description']]
df['finish'] = [i if i in ['Matte', 'Natural', 'Radiant'] else float('Nan') for i in df['finish']]

#Formulation
df['formulation'] = [i.split('Formulation:')[1].split(' ')[1] if 'Formulation:' in i else float('Nan') for i in df['description']]

x = [i if 'Skin Type:' in i else '' for i in df['description']]

df['normal_skin'] = [True if "Normal" in i else False for i in x]
df['oily_skin'] = [True if "Oily" in i else False for i in x]
df['combo_skin'] = [True if 'Combination' in i else False for i in x]
df['dry_skin'] = [True if 'Dry' in i else False for i in x]

#SPF
df['SPF'] = ["SPF" in i for i in df['name']]


Extract the information from the description column eg. Coverage type, finish, formulation and spf. 

In [55]:
df.dtypes

brand                    object
name                     object
brand_id                  int64
brand_image_url          object
product_id               object
product_image_url        object
rating                    int64
skin_type                object
eye_color                object
skin_concerns            object
incentivized_review      object
skin_tone                object
age                      object
beauty_insider           object
user_name                object
review_text              object
price                    object
recommended              object
first_submission_date    object
last_submission_date     object
location                 object
description              object
coverage                 object
finish                   object
normal_skin                bool
oily_skin                  bool
combo_skin                 bool
dry_skin                   bool
SPF                        bool
formulation              object
dtype: object

In [58]:
#reformat to float and get rid of price ranges and dollar signs
df['price'] = df['price'].astype(str)
df['price'] = [i[-6:] for i in df['price']]
df['price'] = [float(i.replace("$", "")) for i in df['price']]

#change submission time data to datetime
df['first_submission_date'] = [datetime.datetime.strptime(i[:19], '%Y-%m-%dT%H:%M:%S') for i in df['first_submission_date']]
df['last_submission_date'] = [datetime.datetime.strptime(i[:19], '%Y-%m-%dT%H:%M:%S') for i in df['last_submission_date']]


In [60]:
df_export = df[['brand', 'name', 'product_id', 'coverage','finish', 'formulation','normal_skin','oily_skin', 'combo_skin',
               'dry_skin', 'SPF', 'user_name', 'rating', 'recommended', 'skin_type', 'eye_color', 'skin_tone', 'skin_concerns',
                'age', 'beauty_insider', 'price','first_submission_date', 'last_submission_date', 'location', 'incentivized_review', 
               'review_text', 'brand_image_url', 'product_image_url']]
df_export.to_csv('C:/Users/Jackson/Documents/Data Science/Sephora Scraper_Attempts/sephora_reviews_cleaned.csv', index = False)